In [13]:
import matplotlib.pyplot as plt
import pandas as pd
from pymeos import *
from tqdm.notebook import tqdm
import contextily as cx
from pymeos.db.psycopg import MobilityDB
from datetime import datetime, timedelta 
import numpy as np
from shapely.geometry import Point

tqdm.pandas()

pymeos_initialize()

host = "localhost"
port = 5432
db = "mobilitydb"
user = "postgres"
password = "postgres"

connection = MobilityDB.connect(
    host=host, port=port, dbname=db, user=user, password=password
)
cursor = connection.cursor()

In [14]:
cursor.execute(f"SELECT MMSI FROM public.PyMEOS_demo;")
mmsi_list = cursor.fetchall()

rows={}
for mmsi in mmsi_list:
    ship_mmsi = mmsi[0]
    cursor.execute(f"SELECT * FROM public.PyMEOS_demo WHERE MMSI = {ship_mmsi} ;")
    _, trajectory, sog = cursor.fetchone()
    rows[mmsi] = trajectory

In [15]:
start_date = datetime(2023, 6, 1, 0, 0, 0)
time_delta = timedelta(minutes=1)
time_ranges = [start_date + i * time_delta for i in range(1440)]
time_ranges

[datetime.datetime(2023, 6, 1, 0, 0),
 datetime.datetime(2023, 6, 1, 0, 1),
 datetime.datetime(2023, 6, 1, 0, 2),
 datetime.datetime(2023, 6, 1, 0, 3),
 datetime.datetime(2023, 6, 1, 0, 4),
 datetime.datetime(2023, 6, 1, 0, 5),
 datetime.datetime(2023, 6, 1, 0, 6),
 datetime.datetime(2023, 6, 1, 0, 7),
 datetime.datetime(2023, 6, 1, 0, 8),
 datetime.datetime(2023, 6, 1, 0, 9),
 datetime.datetime(2023, 6, 1, 0, 10),
 datetime.datetime(2023, 6, 1, 0, 11),
 datetime.datetime(2023, 6, 1, 0, 12),
 datetime.datetime(2023, 6, 1, 0, 13),
 datetime.datetime(2023, 6, 1, 0, 14),
 datetime.datetime(2023, 6, 1, 0, 15),
 datetime.datetime(2023, 6, 1, 0, 16),
 datetime.datetime(2023, 6, 1, 0, 17),
 datetime.datetime(2023, 6, 1, 0, 18),
 datetime.datetime(2023, 6, 1, 0, 19),
 datetime.datetime(2023, 6, 1, 0, 20),
 datetime.datetime(2023, 6, 1, 0, 21),
 datetime.datetime(2023, 6, 1, 0, 22),
 datetime.datetime(2023, 6, 1, 0, 23),
 datetime.datetime(2023, 6, 1, 0, 24),
 datetime.datetime(2023, 6, 1, 0, 2

In [6]:
traj = rows[(9112856,)]
traj.timestamps()

[datetime.datetime(2023, 6, 1, 0, 0, tzinfo=tzoffset(None, 7200)),
 datetime.datetime(2023, 6, 1, 0, 1, 42, tzinfo=tzoffset(None, 7200)),
 datetime.datetime(2023, 6, 1, 0, 4, 42, tzinfo=tzoffset(None, 7200)),
 datetime.datetime(2023, 6, 1, 0, 6, 2, tzinfo=tzoffset(None, 7200)),
 datetime.datetime(2023, 6, 1, 0, 10, 42, tzinfo=tzoffset(None, 7200)),
 datetime.datetime(2023, 6, 1, 0, 14, tzinfo=tzoffset(None, 7200)),
 datetime.datetime(2023, 6, 1, 0, 16, 2, tzinfo=tzoffset(None, 7200)),
 datetime.datetime(2023, 6, 1, 0, 22, 2, tzinfo=tzoffset(None, 7200)),
 datetime.datetime(2023, 6, 1, 0, 37, 2, tzinfo=tzoffset(None, 7200)),
 datetime.datetime(2023, 6, 1, 0, 40, 2, tzinfo=tzoffset(None, 7200)),
 datetime.datetime(2023, 6, 1, 0, 43, 2, tzinfo=tzoffset(None, 7200)),
 datetime.datetime(2023, 6, 1, 0, 53, 2, tzinfo=tzoffset(None, 7200)),
 datetime.datetime(2023, 6, 1, 1, 8, 10, tzinfo=tzoffset(None, 7200)),
 datetime.datetime(2023, 6, 1, 1, 14, 2, tzinfo=tzoffset(None, 7200)),
 datetime.dat

In [7]:
traj_resampled = traj.temporal_sample(start=time_ranges[0],duration= timedelta(minutes=1))
traj_resampled.timestamps()

[datetime.datetime(2023, 6, 1, 0, 0, tzinfo=tzoffset(None, 7200)),
 datetime.datetime(2023, 6, 1, 0, 1, tzinfo=tzoffset(None, 7200)),
 datetime.datetime(2023, 6, 1, 0, 2, tzinfo=tzoffset(None, 7200)),
 datetime.datetime(2023, 6, 1, 0, 3, tzinfo=tzoffset(None, 7200)),
 datetime.datetime(2023, 6, 1, 0, 4, tzinfo=tzoffset(None, 7200)),
 datetime.datetime(2023, 6, 1, 0, 5, tzinfo=tzoffset(None, 7200)),
 datetime.datetime(2023, 6, 1, 0, 6, tzinfo=tzoffset(None, 7200)),
 datetime.datetime(2023, 6, 1, 0, 7, tzinfo=tzoffset(None, 7200)),
 datetime.datetime(2023, 6, 1, 0, 8, tzinfo=tzoffset(None, 7200)),
 datetime.datetime(2023, 6, 1, 0, 9, tzinfo=tzoffset(None, 7200)),
 datetime.datetime(2023, 6, 1, 0, 10, tzinfo=tzoffset(None, 7200)),
 datetime.datetime(2023, 6, 1, 0, 11, tzinfo=tzoffset(None, 7200)),
 datetime.datetime(2023, 6, 1, 0, 12, tzinfo=tzoffset(None, 7200)),
 datetime.datetime(2023, 6, 1, 0, 13, tzinfo=tzoffset(None, 7200)),
 datetime.datetime(2023, 6, 1, 0, 14, tzinfo=tzoffset(None

In [16]:
start_time = traj_resampled.start_timestamp().replace(tzinfo=None)
end_time = traj_resampled.end_timestamp().replace(tzinfo=None)
print(f"Start time: {start_time}")
print(f"End time: {end_time}")


Start time: 2023-06-01 00:00:00
End time: 2023-06-01 23:57:00


In [14]:
print(len(time_ranges))
print(len(traj_resampled.timestamps()))

1440
1438


In [18]:
start_index = time_ranges.index(start_time)
end_index = time_ranges.index(end_time)
print(start_index)
print(end_index)

0
1437


In [27]:

empty_point_wkt = Point().wkt  # "POINT EMPTY"

matrix = np.full(1440, empty_point_wkt, dtype=object)



In [30]:

values= traj_resampled.values()
for i in range(start_index, end_index + 1):
    matrix[i] = values[i].wkt

In [31]:
matrix

array(['POINT (12.3227 56.1102)', 'POINT (12.3227 56.1102)',
       'POINT (12.322700000165371 56.11018999999999)', ...,
       'POINT (12.322700001519749 56.110176086956514)', 'POINT EMPTY',
       'POINT EMPTY'], dtype=object)

# Creating the matrix

In [9]:
%%time 

import matplotlib.pyplot as plt
import pandas as pd
from pymeos import *
from tqdm.notebook import tqdm
import contextily as cx
from pymeos.db.psycopg import MobilityDB
from datetime import datetime, timedelta 
import numpy as np
from shapely.geometry import Point

tqdm.pandas()

pymeos_initialize()

host = "localhost"
port = 5432
db = "mobilitydb"
user = "postgres"
password = "postgres"

connection = MobilityDB.connect(
    host=host, port=port, dbname=db, user=user, password=password
)
cursor = connection.cursor()

cursor.execute(f"SELECT MMSI FROM public.PyMEOS_demo;")
mmsi_list = cursor.fetchall()

rows={}
for mmsi in mmsi_list:
    ship_mmsi = mmsi[0]
    cursor.execute(f"SELECT * FROM public.PyMEOS_demo WHERE MMSI = {ship_mmsi} ;")
    _, trajectory, sog = cursor.fetchone()
    rows[mmsi] = trajectory


CPU times: user 10.1 s, sys: 584 ms, total: 10.7 s
Wall time: 15.7 s


In [56]:
%%time
import numpy as np
from tqdm.notebook import tqdm
start_date = datetime(2023, 6, 1, 0, 0, 0)
time_delta = timedelta(minutes=1)
time_ranges = [start_date + i * time_delta for i in range(1440)]

mmsi_index = {}
empty_point_wkt = Point().wkt  # "POINT EMPTY"
empty_point_wkt


        


CPU times: user 367 µs, sys: 55 µs, total: 422 µs
Wall time: 438 µs


'POINT EMPTY'

In [59]:
matrix = np.full((len(mmsi_list), 1440), empty_point_wkt, dtype=object)
#size of matrix in mb
matrix.nbytes / 1024 / 1024

63.951416015625

In [96]:
for i, mmsi in enumerate(tqdm(mmsi_list)):
    traj = rows[mmsi]
    num_points = traj.num_instants()
    if num_points == 2:
        print(f"Trajectory with only two points:{i} - {mmsi}")
        continue
    


  0%|          | 0/5821 [00:00<?, ?it/s]

Trajectory with only two points:16 - (111257010,)
Trajectory with only two points:40 - (205435790,)
Trajectory with only two points:62 - (209217000,)
Trajectory with only two points:63 - (209221000,)
Trajectory with only two points:65 - (209247000,)
Trajectory with only two points:89 - (209708000,)
Trajectory with only two points:115 - (210185000,)
Trajectory with only two points:136 - (210928545,)
Trajectory with only two points:153 - (211112820,)
Trajectory with only two points:173 - (211127710,)
Trajectory with only two points:178 - (211131000,)
Trajectory with only two points:183 - (211139720,)
Trajectory with only two points:191 - (211149270,)
Trajectory with only two points:196 - (211155540,)
Trajectory with only two points:205 - (211165590,)
Trajectory with only two points:232 - (211185920,)
Trajectory with only two points:234 - (211186950,)
Trajectory with only two points:241 - (211189180,)
Trajectory with only two points:272 - (211205210,)
Trajectory with only two points:283 -

In [105]:
traj = rows[mmsi_list[40]]
print(mmsi_list[0])
print(traj.num_instants())

(5322,)
2


In [106]:
traj

TGeogPointSeq([POINT(5.71542 50.8817)@2023-06-01 04:25:39+02, POINT(5.71542 50.8817)@2023-06-01 04:28:39+02])

In [107]:
traj = traj.temporal_precision(timedelta(minutes=1))
traj

TGeogPointSeq([POINT(5.71542 50.8817)@2023-06-01 04:25:00+02, POINT(5.71542 50.8817)@2023-06-01 04:28:00+02])

In [108]:
start = traj.start_timestamp()
start            

datetime.datetime(2023, 6, 1, 4, 25, tzinfo=tzoffset(None, 7200))

In [82]:
single_timestamp = traj.timestamps()[0].replace(tzinfo=None)

In [83]:
index = time_ranges.index(single_timestamp)

In [109]:
traj_resampled = traj.temporal_sample(start=time_ranges[0],duration= timedelta(minutes=1))
traj_resampled

TGeogPointSeq({POINT(5.71542 50.8817)@2023-06-01 04:25:00+02, POINT(5.71542 50.8817)@2023-06-01 04:26:00+02, POINT(5.71542 50.8817)@2023-06-01 04:27:00+02, POINT(5.71542 50.8817)@2023-06-01 04:28:00+02})

In [110]:
start_time = traj_resampled.start_timestamp().replace(tzinfo=None)
end_time = traj_resampled.end_timestamp().replace(tzinfo=None)
start_index = time_ranges.index(start_time)
end_index = time_ranges.index(end_time)

In [111]:
len(traj_resampled.timestamps())

4

In [112]:
trajectory_array = np.array([point.wkt for point in traj_resampled.values()])
trajectory_array

array(['POINT (5.71542 50.8817)', 'POINT (5.71542 50.8817)',
       'POINT (5.71542 50.8817)', 'POINT (5.71542 50.8817)'], dtype='<U23')

In [116]:
matrix[40,start_index:end_index+1]

array(['POINT (5.71542 50.8817)', 'POINT (5.71542 50.8817)',
       'POINT (5.71542 50.8817)', 'POINT (5.71542 50.8817)'], dtype=object)

In [52]:
start_index 

927

In [114]:

matrix[40, start_index:end_index+1] = trajectory_array

In [ ]:
for j in range(start_index, end_index ):
    # update the matrix with the point
    matrix[i][j] = values[j-start_index].wkt

In [ ]:
for i in tqdm(range(len(mmsi_list))):
    try:
        mmsi_index[mmsi_list[i]] = i
        traj = rows[mmsi_list[i]]
        if traj.num_instants() > 3:
            traj = traj.temporal_precision(timedelta(minutes=1))
            start = traj.start_timestamp()
            traj_resampled = traj.temporal_sample(start=start,duration= timedelta(minutes=1))
            #traj_resampled.timestamps()
            start_time = traj_resampled.start_timestamp().replace(tzinfo=None)
            end_time = traj_resampled.end_timestamp().replace(tzinfo=None)
            # print(f"Start time: {start_time}")
            # print(f"End time: {end_time}")
            start_index = time_ranges.index(start_time)
            end_index = time_ranges.index(end_time)
            # print(start_index)
            # print(end_index)
            values= traj_resampled.values()

            for j in range(start_index, end_index ):
                # update the matrix with the point
                matrix[i][j] = values[j-start_index].wkt
    except Exception as e:
        print(f"Error: {e} for MMSI: {mmsi_list[i]}")

In [135]:
matrix

array([['POINT (12.3227 56.1102)',
        'POINT (12.322700000082685 56.1101985)',
        'POINT (12.322700000440992 56.110173333333336)', ...,
        'POINT EMPTY', 'POINT EMPTY', 'POINT EMPTY'],
       ['POINT EMPTY', 'POINT EMPTY', 'POINT EMPTY', ...,
        'POINT (10.5245 54.85059997395833)', 'POINT EMPTY',
        'POINT EMPTY']], dtype=object)

In [137]:
for mmsi in mmsi_list:
    i = mmsi_index[mmsi]
    t_index = timestamps.index(datetime(2023, 6, 1, 3, 0, 0))
    print(f"Time: {timestamps[t_index]}")
    print(f"Point: {matrix[i][t_index]}")

Time: 2023-06-01 03:00:00
Point: POINT (12.322672455089819 56.11010000000384)
Time: 2023-06-01 03:00:00
Point: POINT (10.8616 55.0625)


In [146]:
from pympler import asizeof
size_in_bytes = asizeof.asizeof(matrix)
size_in_megabytes = size_in_bytes / (1024 * 1024)
print(f"Size of the matrix: {size_in_megabytes} MB")

Size of the matrix: 63.95155334472656 MB


In [151]:
# dimension of matrix
matrix.shape

#Get columns 0
ts_1 =matrix[:, 0]
ts_1

array(['POINT EMPTY', 'POINT (8.42333 55.4718)', 'POINT EMPTY', ...,
       'POINT EMPTY', 'POINT EMPTY', 'POINT EMPTY'], dtype=object)

In [154]:
# Create a list from ts_1 with all the points that are not empty
# using numpy operations
points = ts_1[ts_1 != empty_point_wkt]
points

array(['POINT (8.42333 55.4718)', 'POINT (12.3227 56.1102)',
       'POINT (8.565365280701755 55.08710000000018)', ...,
       'POINT (8.989036848640897 53.85318377997052)',
       'POINT (7.112356 53.33180000047196)', 'POINT (10.619 55.0587)'],
      dtype=object)

# Fully working example

In [ ]:

import matplotlib.pyplot as plt
import pandas as pd
from pymeos import *
from pymeos.db.psycopg import MobilityDB
from datetime import datetime, timedelta 
import numpy as np
from shapely.geometry import Point

pymeos_initialize()

host = "localhost"
port = 5432
db = "mobilitydb"
user = "postgres"
password = "postgres"

connection = MobilityDB.connect(
    host=host, port=port, dbname=db, user=user, password=password
)
cursor = connection.cursor()

cursor.execute(f"SELECT MMSI FROM public.PyMEOS_demo;")
mmsi_list = cursor.fetchall()

rows={}
for mmsi in mmsi_list:
    ship_mmsi = mmsi[0]
    cursor.execute(f"SELECT * FROM public.PyMEOS_demo WHERE MMSI = {ship_mmsi} ;")
    _, trajectory, sog = cursor.fetchone()
    rows[mmsi] = trajectory

from tqdm import tqdm
start_date = datetime(2023, 6, 1, 0, 0, 0)
time_delta = timedelta(minutes=1)
time_ranges = [start_date + i * time_delta for i in range(1440)]

mmsi_index = {}
empty_point_wkt = Point().wkt  # "POINT EMPTY"
#Create a numpy matrix of size 1440x len(mmsi_list) with empty points
matrix = np.full((len(mmsi_list), 1440), empty_point_wkt, dtype=object)

for i in tqdm(range(len(mmsi_list))):
    try:
        mmsi_index[mmsi_list[i]] = i
        traj = rows[mmsi_list[i]]
        if traj.num_instants() > 3:
            traj = traj.temporal_precision(timedelta(minutes=1))
            start = traj.start_timestamp()
            traj_resampled = traj.temporal_sample(start=start,duration= timedelta(minutes=1))
            #traj_resampled.timestamps()
            start_time = traj_resampled.start_timestamp().replace(tzinfo=None)
            end_time = traj_resampled.end_timestamp().replace(tzinfo=None)
            # print(f"Start time: {start_time}")
            # print(f"End time: {end_time}")
            start_index = time_ranges.index(start_time)
            end_index = time_ranges.index(end_time)
            # print(start_index)
            # print(end_index)
            values= traj_resampled.values()

            for j in range(start_index, end_index ):
                # update the matrix with the point
                matrix[i][j] = values[j-start_index].wkt
    except Exception as e:
        print(f"Error: {e} for MMSI: {mmsi_list[i]}")
        


vlayer = QgsVectorLayer("Point", "MobilityBD Data", "memory")
pr = vlayer.dataProvider()
pr.addAttributes([QgsField("time", QVariant.DateTime)])
vlayer.updateFields()
tp = vlayer.temporalProperties()
tp.setIsActive(True)
tp.setMode(qgis.core.QgsVectorLayerTemporalProperties.ModeFeatureDateTimeInstantFromField)
tp.setStartField("time")
vlayer.updateFields()

QgsProject.instance().addMapLayer(vlayer)

ts_1 =matrix[:, 0]
points = ts_1[ts_1 != empty_point_wkt]

qgis_fields_list = []
for wkt in np.nditer(points, flags=['refs_ok']):
    feat = QgsFeature(vlayer.fields())
    feat.setAttributes([datetime_obj])  # Set its attributes

    # Create geometry from WKT string
    geom = QgsGeometry.fromWkt(wkt.item())
    feat.setGeometry(geom)  # Set its geometry
    qgis_fields_list.append(feat)

vlayer.startEditing()
vlayer.addFeatures(qgis_fields_list) # Add list of features to vlayer
vlayer.commitChanges()
iface.vectorLayerTools().stopEditing(self.vlayer)


# Attempt at CUPY

In [3]:
%%time
import matplotlib.pyplot as plt
import pandas as pd
from pymeos import *
from pymeos.db.psycopg import MobilityDB
from datetime import datetime, timedelta 
import numpy as np
from shapely.geometry import Point



host = "localhost"
port = 5432
db = "mobilitydb"
user = "postgres"
password = "postgres"

connection = MobilityDB.connect(
    host=host, port=port, dbname=db, user=user, password=password
)
cursor = connection.cursor()

cursor.execute(f"SELECT MMSI FROM public.PyMEOS_demo;")
mmsi_list = cursor.fetchall()

rows = {}
for mmsi in mmsi_list:
    ship_mmsi = mmsi[0]
    cursor.execute(f"SELECT * FROM public.PyMEOS_demo WHERE MMSI = {ship_mmsi} ;")
    _, trajectory, sog = cursor.fetchone()
    rows[mmsi] = trajectory

start_date = datetime(2023, 6, 1, 0, 0, 0)
time_delta = timedelta(minutes=1)
time_ranges = [start_date + i * time_delta for i in range(1440)]

mmsi_index = {}
empty_point_wkt = "POINT EMPTY"  # Use a string for empty point representation
# Create a CuPy matrix of size 1440 x len(mmsi_list) with empty points
matrix = cp.full((len(mmsi_list), 1440), empty_point_wkt, dtype=object)

for i in range(len(mmsi_list)):
    try:
        mmsi_index[mmsi_list[i][0]] = i
        traj = rows[mmsi_list[i][0]]
        if traj.num_instants() > 3:
            traj = traj.temporal_precision(timedelta(minutes=1))
            start = traj.start_timestamp()
            traj_resampled = traj.temporal_sample(start=start, duration=timedelta(minutes=1))
            start_time = traj_resampled.start_timestamp().replace(tzinfo=None)
            end_time = traj_resampled.end_timestamp().replace(tzinfo=None)
            start_index = time_ranges.index(start_time)
            end_index = time_ranges.index(end_time)
            values = traj_resampled.values()

            for j in range(start_index, end_index):
                # update the matrix with the point
                matrix[i, j] = values[j - start_index].wkt
    except Exception as e:
        pass

# Convert CuPy matrix back to NumPy if needed
numpy_matrix = cp.asnumpy(matrix)



AttributeError: 'str' object has no attribute 'size'

In [1]:
import cupy as cp

In [2]:
cp.random.randint(0, 100, size = (5000, 1440))
#matrix = cp.full((len(mmsi_list), 1440), empty_point_wkt, dtype=object)

array([[ 8, 79, 45, ..., 57, 47, 65],
       [74, 40,  0, ..., 16, 14, 88],
       [41, 62, 94, ..., 53, 48, 57],
       ...,
       [54, 60, 42, ..., 92,  1, 48],
       [44,  6, 20, ..., 77, 90, 59],
       [ 5, 70, 99, ..., 74, 51, 40]])

In [6]:
import cupy as cp

# Create a large matrix on the GPU using CuPy
matrix_size = (10000, 10000)  # Example large matrix size
matrix = cp.random.rand(*matrix_size).astype(cp.float32)





In [8]:
%%timeit
result_matrix_host = cp.asnumpy(matrix)


45.1 ms ± 1.41 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [3]:
dir(cp)

['AxisError',
 'ComplexWarning',
 'DataSource',
 'ElementwiseKernel',
 'Inf',
 'Infinity',
 'ModuleDeprecationWarning',
 'NAN',
 'NINF',
 'NZERO',
 'NaN',
 'PINF',
 'PZERO',
 'RankWarning',
 'RawKernel',
 'RawModule',
 'ReductionKernel',
 'TooHardError',
 'VisibleDeprecationWarning',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__getattr__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 '_binary',
 '_core',
 '_creation',
 '_cupy',
 '_cupyx',
 '_default_memory_pool',
 '_default_pinned_memory_pool',
 '_deprecated_apis',
 '_embed_signatures',
 '_environment',
 '_functional',
 '_functools',
 '_indexing',
 '_io',
 '_logic',
 '_manipulation',
 '_math',
 '_misc',
 '_numpy',
 '_padding',
 '_sorting',
 '_statistics',
 '_sys',
 '_util',
 '_version',
 'abs',
 'absolute',
 'add',
 'all',
 'allclose',
 'alltrue',
 'amax',
 'amin',
 'angle',
 'any',
 'append',
 'apply_along_axis',
 'arange',
 'arccos',
 'arccosh',
 'arcsin',
 'arcsinh',
 'arctan

# Debugging optimized matrix

In [5]:
from pymeos.db.psycopg import MobilityDB

from pymeos import *
from datetime import datetime, timedelta
import time
from collections import deque
from pympler import asizeof
import gc
from enum import Enum
import numpy as np
from shapely.geometry import Point

LEN_DEQUEUE_FPS = 5 # Length of the dequeue to calculate the average FPS
LEN_DEQUEUE_BUFFER = 2 # Length of the dequeue to keep the keys to keep in the buffer


PERCENTAGE_OF_OBJECTS = 0.5 # To not overload the memory, we only take a percentage of the ships in the database
FRAMES_PER_TIME_DELTA = 120 # Number of frames associated to one Time delta


In [6]:


class MobilityDB_Database:
    """
    Singleton class used to connect to the MobilityDB database.
    """
    
    def __init__(self):
        connection_params = {
        "host": "localhost",
        "port": 5432,
        "dbname": "mobilitydb",
        "user": "postgres",
        "password": "postgres"
        }
        try: 
            self.table_name = "PyMEOS_demo"
            self.id_column_name = "MMSI"
            self.tpoint_column_name = "trajectory"            
            self.SRID = 4326            
            self.connection = MobilityDB.connect(**connection_params)

            self.cursor = self.connection.cursor()

            self.cursor.execute(f"SELECT {self.id_column_name} FROM public.{self.table_name};")
            self.ids_list = self.cursor.fetchall()
        except Exception as e:
            pass

    def get_subset_of_ids(self, percentage=0.001):
        """
        Returns a subset of the objects ids in the table, based on the given percentage.
        """
        return self.ids_list[:int(len(self.ids_list)*percentage)]

    def get_subset_of_tpoints(self, ids_list):
        """
        For each object in the ids_list :
            Fetch the subset of the associated Tpoints between the start and end timestamps
            contained in the STBOX defined by the xmin, ymin, xmax, ymax.
        """
        try:
            rows={}
            for id in ids_list:
                object_id = id[0]
                self.cursor.execute(f"""SELECT {self.tpoint_column_name} 
                               FROM public.{self.table_name} 
                               WHERE {self.id_column_name} = {object_id} ;""")
                subset_tpoint = self.cursor.fetchone()
                if subset_tpoint[0]:
                    rows[id] = subset_tpoint[0]

            return rows
        except Exception as e:
            pass

    def get_min_timestamp(self):
        """
        Returns the min timestamp of the tpoints columns.

        """
        try:
            
            self.cursor.execute(f"SELECT MIN(startTimestamp({self.tpoint_column_name})) AS earliest_timestamp FROM public.{self.table_name};")
            return self.cursor.fetchone()[0]
        except Exception as e:
            pass

    def get_max_timestamp(self):
        """
        Returns the max timestamp of the tpoints columns.

        """
        try:
            self.cursor.execute(f"SELECT MAX(endTimestamp({self.tpoint_column_name})) AS latest_timestamp FROM public.{self.table_name};")
            return self.cursor.fetchone()[0]
        except Exception as e:
            pass


    def close(self):
        """
        Close the connection to the MobilityDB database.
        """
        self.cursor.close()
        self.connection.close()


In [7]:
db = MobilityDB_Database()
pymeos_initialize()
        
ids_list = db.get_subset_of_ids(PERCENTAGE_OF_OBJECTS) 
        


In [22]:
start_date = db.get_min_timestamp().replace(tzinfo=None)
end_date = db.get_max_timestamp().replace(tzinfo=None)
total_frames = (end_date - start_date) // timedelta(minutes=1)


timestamps = [start_date + i * timedelta(minutes=1) for i in range(total_frames)]
timestamps_strings = [dt.strftime('%Y-%m-%d %H:%M:%S') for dt in timestamps]

In [8]:

len(ids_list)

2910

In [23]:
time_ranges = timestamps

In [10]:
rows =db.get_subset_of_tpoints(ids_list)
#print(f"Number of rows : {len(rows)}")
time_ranges = timestamps
# remove timezones for all the timestamps



empty_point_wkt = Point().wkt  # "POINT EMPTY"
#Create a numpy matrix of size 1440x len(mmsi_list) with empty points
matrix = np.full((len(ids_list), 1440), empty_point_wkt, dtype=object)

In [11]:
i = 0
object_id = ids_list[i]

In [13]:
traj = rows[object_id]
traj = traj.temporal_precision(timedelta(minutes=1)) # precision is 1 minute so Two points in a minute will become 1
num_instants = traj.num_instants()

In [14]:
traj_resampled = traj.temporal_sample(start=time_ranges[0],duration= timedelta(minutes=1))

traj_resampled

TGeogPointSeq({POINT(12.605989285714285 55.6845)@2023-06-01 01:06:00+02, POINT(12.606 55.6845)@2023-06-01 01:07:00+02, POINT(12.606011666666667 55.68443166666667)@2023-06-01 01:08:00+02, POINT(12.606043333333334 55.684475)@2023-06-01 01:09:00+02, POINT(12.605921666666667 55.684583333333336)@2023-06-01 01:10:00+02, POINT(12.605835 55.684356666666666)@2023-06-01 01:11:00+02, POINT(12.6058 55.6843)@2023-06-01 01:12:00+02, POINT(12.6058 55.6843)@2023-06-01 01:13:00+02, POINT(12.6058 55.6843)@2023-06-01 01:14:00+02, POINT(12.6058 55.6843)@2023-06-01 01:15:00+02, POINT(12.6058 55.6843)@2023-06-01 01:16:00+02, POINT(12.6058 55.6843)@2023-06-01 01:17:00+02, POINT(12.6058 55.6843)@2023-06-01 01:18:00+02, POINT(12.6058 55.6843)@2023-06-01 01:19:00+02, POINT(12.6058 55.6843)@2023-06-01 01:20:00+02, POINT(12.6058 55.6843)@2023-06-01 01:21:00+02, POINT(12.6058 55.6843)@2023-06-01 01:22:00+02, POINT(12.6058 55.6843)@2023-06-01 01:23:00+02, POINT(12.6058 55.6843)@2023-06-01 01:24:00+02, POINT(12.6058

In [16]:
traj_resampled.start_timestamp().replace(tzinfo=None)

datetime.datetime(2023, 6, 1, 1, 6)

In [24]:
time_ranges.index(traj_resampled.start_timestamp().replace(tzinfo=None))

66

In [15]:
start_index = time_ranges.index(  )
end_index = time_ranges.index( traj_resampled.end_timestamp().replace(tzinfo=None) )


ValueError: datetime.datetime(2023, 6, 1, 1, 6) is not in list

In [ ]:

trajectory_array = np.array([point.wkt for point in traj_resampled.values()])

matrix[i, start_index:end_index+1] = trajectory_array
print(f"matrix updated from index {start_index} to {end_index} for object_id {object_id}")

In [134]:
len(timestamps)

1439

In [139]:
len(ids_list)

2910

In [140]:
rows = db.get_subset_of_tpoints(ids_list)
len(rows)

2910

In [141]:
rows={}
for id in tt.data.ids_list:
    object_id = id[0]
    tt.data.db.cursor.execute(f"SELECT {tt.data.db.tpoint_column_name} FROM public.{tt.data.db.table_name} WHERE {tt.data.db.id_column_name} = {object_id} ;")
    subset_tpoint = tt.data.db.cursor.fetchone()
    rows[id] = subset_tpoint[0]

NameError: name 'tt' is not defined

In [ ]:
#find first element of matrix that is different from empty_point_wkt
def find_first_element(matrix, empty_point_wkt):
    for i in range(len(matrix)):
        if matrix[i] != empty_point_wkt:
            return i
    return -1

import numpy as np

# Assuming you have defined the matrix and empty_point_wkt variables

# Find elements in the matrix that are not equal to empty_point_wkt
non_empty_elements = np.any(matrix != empty_point_wkt)

# Print the result
print(non_empty_elements)